In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [3]:
(x_train, y_train), (x_text, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_text.reshape(-1, 28, 28, 1) / 255.0

In [5]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

c:\Users\ma310770\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9097 - loss: 0.2941 - val_accuracy: 0.9859 - val_loss: 0.0425
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9856 - loss: 0.0465 - val_accuracy: 0.9885 - val_loss: 0.0369
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9905 - loss: 0.0297 - val_accuracy: 0.9892 - val_loss: 0.0312
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9936 - loss: 0.0198 - val_accuracy: 0.9912 - val_loss: 0.0273
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9954 - loss: 0.0149 - val_accuracy: 0.9909 - val_loss: 0.0301


In [8]:
import json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_weights.weights.h5")

In [1]:
import h5py
import json
import os
import numpy as np

def hdf5_to_chunks(filename, chunk_size=1000, output_dir='chunks'):
    os.makedirs(output_dir, exist_ok=True)
    with h5py.File(filename, 'r') as f:
        lua_table = {}

        def recursively_extract_data(name, obj):
            if isinstance(obj, h5py.Dataset):
                data = obj[()]
                if isinstance(data, np.ndarray):
                    data_list = data.tolist()
                    lua_table[name] = data_list

        f.visititems(recursively_extract_data)

        for i, (key, data) in enumerate(lua_table.items()):
            chunk_filename = os.path.join(output_dir, f'chunk_{i}.json')
            with open(chunk_filename, 'w') as chunk_file:
                json.dump({key: data}, chunk_file)

hdf5_to_chunks('model_weights.weights.h5')


NameError: name 'np' is not defined